# 2021/10/22
The jupypter notebook is used to test the n_neighbour for the ENN


In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
import pandas as pd
import numpy as np
import sys, os

# Load the data

In [3]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/'
#df=pd.read_csv(os.path.join(input_path, 'total.Megalodon.per_read.prob.bed.gz'),compression='gzip', sep='\t')
df=pd.read_csv(os.path.join(input_path, 'total.test.bed'), sep='\t')
print("Data is loading!")

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

df

Data is loading!


,chr,start,read_id,strand,5hmC_prob,5mC_prob,5C_prob,label
0,KJ477685.1,204,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.435397,0.085058,0.479545,2
1,KJ477685.1,261,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.342189,0.500821,0.156990,2
2,KJ477685.1,261,fe8be0cc-f4c4-49f1-80db-c840a6c7dfe9,-,0.577034,0.057676,0.365290,2
3,KJ477685.1,266,3e547b1a-0a3e-4475-9b70-0cbd7c8c82aa,-,0.065316,0.861569,0.073115,2
4,KJ477685.1,266,fe8be0cc-f4c4-49f1-80db-c840a6c7dfe9,-,0.794235,0.028885,0.176880,2
...,...,...,...,...,...,...,...,...
2492,J02459.1,60,bb826a6c-c88b-45ac-9984-8b9d6bfb2f11,-,0.054875,0.013023,0.932103,0
2493,J02459.1,60,bb91267a-6661-4248-a084-554f231398c1,-,0.030720,0.004392,0.964888,0
2494,J02459.1,60,bb9cee29-89d0-4a68-a4b4-8fe5c000289f,-,0.032703,0.006934,0.960363,0
2495,J02459.1,60,bbee6b4a-acac-4db9-bbb8-2f379918b146,-,0.035734,0.003182,0.961085,0


In [4]:
df['label'].value_counts()

1    999
0    999
2    499
Name: label, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, GridSearchCV
#from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [6]:
#Load the feature
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test'
#df=pd.read_csv(os.path.join(input_path, 'total.Megalodon.per_read.prob.bed.gz'),compression='gzip', sep='\t')
df=pd.read_csv(os.path.join(input_path, 'total.test.bed'), sep='\t')

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

X = df_feature
y = df_class

#Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [7]:
from collections import Counter
print("Before the pipeline:\n y_train:{},\n y_test: {}".format(Counter(y_train), Counter(y_test)))

Before the pipeline:
 y_train:Counter({0: 799, 1: 799, 2: 399}),
 y_test: Counter({0: 200, 1: 200, 2: 100})


## Resampling strategy: ENN

In [ ]:
## Define resampling strategy
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import EditedNearestNeighbours, RandomUnderSampler

### ENN

In [12]:
from imblearn.under_sampling import EditedNearestNeighbours

n_neighbors = [20, 50, 100, 200]
for n in n_neighbors:
    print(n)
    ENN = EditedNearestNeighbours(n_neighbors = n, n_jobs = -1)
    X_resampled, y_resampled = ENN.fit_resample(X_train, y_train)
    print(Counter(y_resampled))

20
Counter({1: 660, 0: 525, 2: 399})
50
Counter({1: 565, 2: 399, 0: 385})
100
Counter({1: 450, 2: 399, 0: 236})
200
Counter({2: 399, 1: 369, 0: 1})


# Whole dataset testing for ENN parameter

In [13]:
input_file='/pod/2/li-lab/Ziwei/Nanopore/results/megalodon2.3.4_guppy5.0.14/total.Megalodon.per_read.prob.bed.gz'
df=pd.read_csv(input_file,compression='gzip', sep='\t')
print("Data is loading!")

#Splitting the data into independent and dependent variables
df_feature = df.loc[:,['5hmC_prob','5mC_prob','5C_prob']].values
df_class = df.loc[:,['label']].values
df_class = np.squeeze(df_class) #Convert the label into 1d-array

Data is loading!


In [14]:
df['label'].value_counts()

1    11627345
0     7368258
2        2077
Name: label, dtype: int64

## Count the training/testing

In [16]:
X = df_feature
y = df_class

#Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [17]:
print("Before the pipeline:\n y_train:{},\n y_test: {}".format(Counter(y_train), Counter(y_test)))

Before the pipeline:
 y_train:Counter({1: 9301876, 0: 5894606, 2: 1662}),
 y_test: Counter({1: 2325469, 0: 1473652, 2: 415})


In [18]:
from imblearn.under_sampling import EditedNearestNeighbours

n_neighbors = [20, 50, 100, 200]
for n in n_neighbors:
    print(n)
    ENN = EditedNearestNeighbours(n_neighbors = n, n_jobs = -1)
    X_resampled, y_resampled = ENN.fit_resample(X_train, y_train)
    print(Counter(y_resampled))

20


MemoryError: Unable to allocate 1.46 GiB for an array with shape (9301876, 21) and data type int64